
Reproducing errors converting a tensorflow model to onnx.

```bash
python -m tf2onnx.convert \
  --opset 17 \
  --saved-model vendor/BirdNET-Analyzer/checkpoints/V2.2/BirdNET_GLOBAL_3K_V2.2_Model/variables/ \
  --output data/models/onnx/test0.onnx
```

Building a few tools in tensorflow from source.
Note these take a long time to build.

```bash
bazel build tensorflow/tools/graph_transforms:summarize_graph
bazel build tensorflow/python/tools:saved_model_cli
```

```bash
~/tensorflow/bazel-bin/tensorflow/tools/graph_transforms/summarize_graph \
    --in_graph=/home/anthony/birdclef-2023/vendor/BirdNET-Analyzer/checkpoints/V2.2/BirdNET_GLOBAL_3K_V2.2_Model/saved_model.pb \
    --print_structure=true


~/tensorflow/bazel-bin/tensorflow/tools/graph_transforms/summarize_graph \
    --in_graph=/home/anthony/birdclef-2023/vendor/BirdNET-Analyzer/checkpoints/V2.2/BirdNET_GLOBAL_3K_V2.2_Model/variables/variables.index \
    --print_structure=true
```

In [1]:
# https://github.com/tensorflow/models/issues/8966
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import (
    convert_variables_to_constants_v2,
)

model_path = "../vendor/BirdNET-Analyzer/checkpoints/V2.2/BirdNET_GLOBAL_3K_V2.2_Model/"

# load the saved_model using low-level API
m = tf.saved_model.load(model_path, tags=["serve"])

2023-02-26 04:21:03.860565: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-26 04:21:03.992468: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-26 04:21:04.672149: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-26 04:21:04.672211: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

# frozen graph using the signature alone

In [2]:
list(m.signatures.keys())

['basic']

In [3]:
! mkdir -p ../data/models/birdnet-frozen-v1

In [4]:
frozen_func = convert_variables_to_constants_v2(m.signatures["basic"])
tf.io.write_graph(
    graph_or_graph_def=frozen_func.graph,
    logdir="./",
    name="../data/models/birdnet-frozen-v1/saved_model.pb",
    as_text=False,
)

2023-02-26 04:21:11.241649: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-02-26 04:21:11.241807: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


'./../data/models/birdnet-frozen-v1/saved_model.pb'

In [ ]:
! ~/tensorflow/bazel-bin/tensorflow/tools/graph_transforms/summarize_graph \
    --in_graph=../data/models/birdnet-frozen-v1/saved_model.pb \
    --print_structure=true | grep rfft -C 3

2023-02-26 04:10:20.351744: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
StatefulPartitionedCall/model/ADVANCED_SPEC1/strided_slice/stack_2 (Const): [], value=Tensor<type: int32 shape: [3] values: 1 1 1>
StatefulPartitionedCall/model/ADVANCED_SPEC1/strided_slice/stack_1 (Const): [], value=Tensor<type: int32 shape: [3] values: 0 0 128>
StatefulPartitionedCall/model/ADVANCED_SPEC1/strided_slice/stack (Const): [], value=Tensor<type: int32 shape: [3] values: 0 0 0>
StatefulPartitionedCall/model/ADVANCED_SPEC1/stft/rfft/fft_length (Const): [], value=Tensor<type: int32 shape: [1] values: 512>
StatefulPartitionedCall/model/ADVANCED_SPEC1/stft/hann_window/sub_1/y (Const): [], value=Tensor<type: int32 shape: [] values: 1>
StatefulPartitionedCall/model/ADVANCED_SPEC1/stft/hann_w

Layers with RFFT are in the ADVANCED_SPEC1 subgraph.
Probably need to insert the ComplexAbs operation around here.

## convert with with input signature?

Not clear if this is a win at all

- https://jimmy-shen.medium.com/how-to-freeze-graph-in-tensorflow-2-x-3a3238c70f19

In [5]:
m.basic.input_signature

(TensorSpec(shape=(None, 144000), dtype=tf.float32, name=None),)

In [16]:
m.signatures["basic"].structured_input_signature

((),
 {'inputs': TensorSpec(shape=(None, 144000), dtype=tf.float32, name='inputs')})

In [20]:
full_model = tf.function(lambda x: m.signatures["basic"](inputs=x))
full_model = full_model.get_concrete_function(m.basic.input_signature[0])
frozen_func = convert_variables_to_constants_v2(full_model)
tf.io.write_graph(
    graph_or_graph_def=frozen_func.graph,
    logdir="../data/models/birdnet-frozen-v2/",
    name="saved_model.pb",
    as_text=False,
)

2023-02-26 04:31:29.781665: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-02-26 04:31:29.781814: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


'../data/models/birdnet-frozen-v2/saved_model.pb'

Summarize

In [23]:
! ~/tensorflow/bazel-bin/tensorflow/tools/graph_transforms/summarize_graph \
    --in_graph=../data/models/birdnet-frozen-v2/saved_model.pb \
    --print_structure=true | tail -n10

2023-02-26 04:32:44.305904: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
StatefulPartitionedCall/StatefulPartitionedCall/model/POST_BN_1/FusedBatchNormV3 (FusedBatchNormV3): [StatefulPartitionedCall/StatefulPartitionedCall/model/POST_CONV_1/Conv2D, StatefulPartitionedCall/StatefulPartitionedCall/model/POST_BN_1/ReadVariableOp, StatefulPartitionedCall/StatefulPartitionedCall/model/POST_BN_1/ReadVariableOp_1, StatefulPartitionedCall/StatefulPartitionedCall/model/POST_BN_1/FusedBatchNormV3/ReadVariableOp, StatefulPartitionedCall/StatefulPartitionedCall/model/POST_BN_1/FusedBatchNormV3/ReadVariableOp_1]
StatefulPartitionedCall/StatefulPartitionedCall/model/POST_ACT_1/Relu (Relu): [StatefulPartitionedCall/StatefulPartitionedCall/model/POST_BN_1/FusedBatchNormV3]
StatefulPar

## keras output

In [ ]:
from tensorflow import keras

pb_model = keras.models.load_model(model_path)
pb_model

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7ff4a42c7190>

In [ ]:
pb_model.model.compile()
pb_model.model.save("../data/models/birdnet-frozen-keras-v1/")

INFO:tensorflow:Assets written to: ../data/models/birdnet-frozen-keras-v1/assets


INFO:tensorflow:Assets written to: ../data/models/birdnet-frozen-keras-v1/assets


In [ ]:
! python -m tf2onnx.convert \
    --keras ../data/models/birdnet-frozen-keras-v1/ \
    --output ../data/models/onnx/test0.onnx